In [1]:
from __future__ import unicode_literals
import matplotlib
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import numpy
import os
matplotlib.rc('font', family='DejaVu Sans')

droot = '/home/brandon/Documents/data/' # absolute paths
ddata = os.path.join(droot,'data/')
dexp = os.path.join(ddata,'skeleton')

din = os.path.join(dexp,'in')
dout = os.path.join(dexp,'out')
dplot = os.path.join(dexp,'plot')
dbin = os.path.join(dexp,'bin')

dintemp = os.path.join(din, 'temperature')
dinthermo = os.path.join(din, 'thermograph')
dinosc = os.path.join(din, 'oscilloscope')
dinspectra = os.path.join(din, 'spectra')
dinintense = os.path.join(din, 'photointensity')
dincurrent = os.path.join(din, 'current')

In [85]:
def prepTemperatureData():
    ### THIS WORKS (BC 2016-05-31)
    # inform the user about the specifics of the processing job
    
    frawtemp = os.path.join(dintemp,os.listdir(dintemp)[0])
    fcleantemp = os.path.join(dout,'temperature_time.csv')
    print "the raw data file is %s" % (frawtemp)
    print("The contents of the file looks like this: ")
    with open(frawtemp,'r') as myfile:
        myfile.readline()
        print "    %s" % (myfile.readline())
        
    # working through the raw data file...
    with open(fcleantemp,'w') as cleanfile:
        # generate a dataset
        dataset = numpy.genfromtxt(frawtemp,delimiter=',',
                                  autostrip=True,invalid_raise=False,skip_header=10,
                                  dtype=(datetime,int,float,float,int))
        # correcting the timestamps for millisecond resolution
        now_seconds = 0
        prev_seconds = 0
        now_us = 0
        prev_us = 0
        delta_us = 0
        ts_us = 0
        for line in dataset:
            try:
                now_seconds = datetime.strptime(line[0], "%Y-%m-%d_%H:%M:%S")
                now_us = line[1] * 1000
                if now_seconds != prev_seconds:
                    prev_seconds = now_seconds
                    prev_us = now_us
                    ts_us = timedelta(microseconds=0)
                else:
                    delta_us = timedelta(microseconds=(now_us - prev_us) % 1000000)
                    ts_us = timedelta(microseconds=prev_us) + delta_us
                    prev_us = ts_us.total_seconds()*1000000
                timestamp = (now_seconds + ts_us).strftime("%Y-%m-%d %H:%M:%S.%f")
                #myfile.write("%s,%.2f,%.2f,%i\n" % (timestamp,line[2],line[3],line[4]))
                cleanfile.write("%s,%.2f,%.2f\n" % (timestamp,line[2],line[3]))
                #print now_seconds.strftime("%Y")
                #print timestamp
            except: pass
    
    print "the clean data file is %s" % (fcleantemp)
    print("The contents of the file looks like: ")
    with open(fcleantemp,'r') as myfile:
        myfile.readline()
        print "    %s" % (myfile.readline())

def plotTemperatureData(setpoint=None):
    ### THIS WORKS (BC 2016-05-31)
    
    fcleantemp = os.path.join(dout,'temperature_time.csv')

    with open(fcleantemp,'r') as myfile:
        print('    ' + myfile.readline())
        myfile.close()
    dataset = numpy.genfromtxt(fcleantemp,delimiter=',',dtype=None)
    xax = []
    yax1 = []
    yax2 = []
    starttime = 0
    for line in dataset:
        try:
            if starttime == 0:
                starttime = datetime.strptime(line[0],"%Y-%m-%d %H:%M:%S.%f")
            dt = (datetime.strptime(line[0],"%Y-%m-%d %H:%M:%S.%f")-starttime)
            xax.append(dt.seconds+dt.microseconds/1000000.)
            yax1.append(line[1])
            yax2.append(line[2])
        except:
            print dt
            print yax1
            print yax2
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(xax,yax1,label='Tsurface',
            linestyle='None',marker='o',markersize=1,color='red',markeredgecolor='red')
    ax.plot(xax,yax2,label='Ttube',
            linestyle='None',marker='o',markersize=1,color='blue',markeredgecolor='blue',)
    if setpoint is not None:
        ax.axhline(setpoint,0,xax[-1],color='black',linestyle='--')
    ax.set_ylim(0,100)
    #ax.set_xlim(0,220)
    plt.title('He jet, PI control @40 degC, 0.5 slpm, 10 kHz')
    ax.set_xlabel('runtime, seconds')
    ax.set_ylabel('temperature, degC')
    ax.legend()
    fig.savefig(os.path.join(dplot,'temperature_history.png'),dpi=150)
    plt.close(fig)

def prepPowerData():
    ### THIS WORKS (BC 2016-05-31)
    powertime = []
    powervoltage = []

    files = sorted(os.listdir(dinosc))
    for f in files:
        filename = os.path.join(dinosc,f)
        t = datetime.strptime(f,"%Y-%m-%d_%H:%M:%S") # the time the datapoint was taken
        data = numpy.genfromtxt(filename,delimiter=',')
        xax = [line[0] for line in data]
        data_v = [line[1] for line in data]
        data_vcap = [line[2] for line in data]
        data_i = [line[3] for line in data]
        voltage = max(data_v)-min(data_v)
        power = getpower(vapp=data_v,vcap=data_vcap,title=str(voltage).split('.')[0])
        powertime.append((t,power))
        powervoltage.append((voltage,power))
    numpy.savetxt(os.path.join(dout,'power_time.csv'),powertime,delimiter=',',fmt='%s %f')
    numpy.savetxt(os.path.join(dout,'power_voltage.csv'),powervoltage,delimiter=',',fmt='%f %f')


def getpower(vapp,vcap,title):
    windowSize = 1
    Liss = create_lissajous(Vapp=vapp,Vcap=vcap,window=windowSize)
    #fig = plt.figure()
    #ax = fig.add_subplot(111)
    #plt.scatter([p[0] for p in Liss], [p[1] for p in Liss])
    area = find_area(Liss[0:1000])
    #plt.title('The area of the Lissajous figure is: ' + str(area))
    #ax.set_xlabel('volts, applied')
    #ax.set_ylabel('volts, capacitor')
    #plt.savefig(dplot+title + '_pwr' + str(area).split('.')[0] + '.png',dpi=150)
    #plt.close(fig)
    watts = int(str(area).split('.')[0])*0.47/1000.
    return(watts)

def moving_average(interval, window_size):
    """Finds the moving average of a dataset over a window size."""
    # algorithm via http://stackoverflow.com/questions/11352047/finding-moving-average-from-data-points-in-python
    window = numpy.ones(int(window_size))/float(window_size)
    return numpy.convolve(interval, window, 'same')

def find_area(array):
    """Find the array of a polygon defined as a set of Cartesian points in an array."""
    # algorithm via http://www.arachnoid.com/area_irregular_polygon/index.html
    a = 0
    ox,oy = array[0]
    for x,y in array[1:]:
        a += (x*oy-y*ox)
        ox,oy = x,y
    return abs(a/2)

def create_lissajous(Vapp,Vcap,window=1):
    """Generates a Lissajous figure from the applied and capacitor voltage oscilloscope traces."""
    #x=[Vapp[i][1] for i in range(0,len(Vapp))] # if your data also includes time
    #y=[Vcap[i][1] for i in range(0,len(Vcap))]
    x = Vapp
    y = Vcap
    if window!= 1:
        x = moving_average(x,window)
        y = moving_average(y,window)
    Liss = []
    for i in range(0,len(Vapp)):
        Liss.append((x[i],y[i]))
    return Liss

def plotPowerData(setpoint=None):
    # PLOTS THE POWER-VOLTAGE RELATIONSHIP - looks good
    data1 = numpy.genfromtxt(os.path.join(dout,'power_voltage.csv'),delimiter=' ')
    fig1 = plt.figure()
    ax = fig1.add_subplot(111)
    ax.plot([item[0] for item in data1],[item[1] for item in data1],
           linestyle='None',marker='o',markerfacecolor='blue')
    ax.set_xlabel('applied potential, V')
    ax.set_ylabel('plasma power, W')
    ax.set_title('plasma power vs applied potential')
    fig1.savefig(os.path.join(dplot,'power_voltage.png'),dpi=150)
    plt.close(fig1)
    
    #PLOTS THE POWER-TIME RELATIONSHIP - looks good, change labels
    data2 = numpy.genfromtxt(os.path.join(dout,'power_time.csv'),delimiter='',dtype=None)
    starttime = datetime.strptime(data2[10][0]+'_'+data2[10][1],"%Y-%m-%d_%H:%M:%S")
    pt = []
    for line in data2[10:]:
        try:
            dt = (datetime.strptime(line[0]+'_'+line[1],"%Y-%m-%d_%H:%M:%S")-starttime)
            pt.append([dt.seconds,line[2]])
        except: pass;
    fig = plt.figure()
    ax = plt.subplot(111)
    colors = ['red']
    labels = ['Power']
    xax = [line[0] for line in pt]
    for i in [1]:
    #for i in range(1,len(dataset[0])-1):
        data = [line[i] for line in pt]
        #if i == 0:
        #    voltage = max(data)-min(data)
        ax.plot(xax[:],data[:],
          linestyle='None',marker='o',markeredgecolor=colors[i-1],color=colors[i-1],markersize=4,
          label=labels[i-1])
    plt.legend()
    plt.ylim(0,35)
    plt.title('He jet, PI control @40 degC, 0.5 slpm, 10 kHz')
    ax.set_xlabel('runtime, seconds')
    ax.set_ylabel('power, watts')
    fig.savefig(os.path.join(dplot,'power_time.png'),dpi=150)
    plt.close(fig)
    
    ################################################################
    ### CO-PLOTS THE POWER AND TEMPERATURE RELATIONSHIP TO TIME  ###
    ################################################################
    data2 = numpy.genfromtxt(os.path.join(dout,'power_time.csv'),delimiter='',dtype=None)
    starttime = datetime.strptime(data2[10][0]+'_'+data2[10][1],"%Y-%m-%d_%H:%M:%S")
    
    fclean_temp = os.path.join(dout,'temperature_time.csv')
    data3 = numpy.genfromtxt(fclean_temp,delimiter=',',dtype=None)
    starttime = datetime.strptime(data3[0][0],"%Y-%m-%d %H:%M:%S.%f")
    #print(data3[:5]) #debugging

    tt = [] ## time-temperature
    for line in data3[10:]:
        #print(line[0])
        ##print(line[1])
        #print(line[2])
        #print(line[3])
        try:
            dt = (datetime.strptime(line[0],"%Y-%m-%d %H:%M:%S.%f")-starttime)
            tt.append([dt.seconds+dt.microseconds/1000000.,line[1],line[2]])
        except:
            pass

    pt = [] ## time-power
    for line in data2[10:]:
        try:
            dt = (datetime.strptime(line[0]+'_'+line[1],"%Y-%m-%d_%H:%M:%S")-starttime)
            pt.append([dt.seconds,line[2]])
        except:
            pass

    fig = plt.figure() # time-temperature
    ax = plt.subplot(111)
    colors = ['red','blue']
    labels = ['Tsurface','Ttube']
    xax = [line[0] for line in tt]
    for i in [1,2]:
    #for i in range(1,len(dataset[0])-1):
        try:
            data = [line[i] for line in tt]
        except:
            pass
        #if i == 0:
        #    voltage = max(data)-min(data)
        ax.plot(xax[:],data[:],
          linestyle='None',marker='o',markeredgecolor=colors[i-1],color=colors[i-1],markersize=1,
          label=labels[i-1])
    plt.legend()
    ax.set_ylim(0,100)
    #ax.set_xlim(0,1000)
    plt.title('He jet, PI control @40 degC, 0.5 slpm, 10 kHz')
    ax.set_xlabel('runtime, seconds')
    ax.set_ylabel('temperature, degC')
    ax2 = ax.twinx() # time-power
    xax2 = [line[0] for line in pt]
    labels = ['Power']
    for i in [1]:
    #for i in range(1,len(dataset[0])-1):
        data = [line[i] for line in pt]
        #if i == 0:
        #    voltage = max(data)-min(data)
        ax2.plot(xax2[:],data[:],
          linestyle='None',marker='o',markeredgecolor='green',color='green',markersize=2,
          label=labels[i-1])
    if setpoint is not None:
        ax.axhline(setpoint,0,xax[-1],color='black',linestyle='--')
    plt.legend(loc=9)
    ax2.set_ylim(0,20)
    #ax2.set_xlim(0,1000)
    ax2.set_ylabel('power,watts')

    fig.savefig(os.path.join(dplot,'temperature_power_time.png'),dpi=150)
    plt.close(fig)

def plotspectra():
    ### plotting spectra
    #$$ important bands: http://www.fch.vut.cz/~krcma/vyuka/plazma/3-oes-en.pdf
    files = sorted(os.listdir(dinspectra))

    ### plot and save the background baseline
    baseline = numpy.genfromtxt(os.path.join(dinspectra,files[-1]),delimiter=',')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot([line[0] for line in baseline],[line[1] for line in baseline])
    ax.set_ylim(0,2000)
    fig.savefig(os.path.join(dplot,'spectra','baseline'),dpi=150)
    plt.close(fig)
    
    ### extract data from each file and save for later
    peaks_time = os.path.join(dout,'peaks_time.csv')
    
    with open(peaks_time,'w') as outfile:
        ### plot and save the background-correct spectra
        for f in sorted(os.listdir(dinspectra)):
            data = numpy.genfromtxt(os.path.join(dinspectra,f),delimiter=',')
            xax = [line[0] for line in data]
            yax = [line[1]-baseline[i][1] for i,line in enumerate(data)]
            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.plot(xax,yax,color='black')

            peaks = {
                'p2_00' : 412, #336.86,
                'p2_01' : 470, #357.13,
                'p2_02' : 535, #379.73,
                'p2_03' : 608, #404.95,
                'p2_04' : 697, #435.46,
                'p2_10' : 352, #315.77,
                'p2_12' : 459, #353.30,
                'p2_13' : 522, #375.22,
                'p2_14' : 591, #399.09,
                'n1_00' : 567, #390.81,
                'n1_01' : 671, #426.57,
                'n1_02' : 799, #470.09,
            }

            timestamp = f[9:]
            peakintensity = ",".join([str(yax[i]) for i in sorted(peaks.values())])
            outfile.write("%s,%s\n" % (timestamp,peakintensity))
            
            #max_wavelength = 0
            #max_intensity = 0
            #for i,intensity in enumerate(yax):
            #    wavelength = xax[i]
            #    if intensity > max_intensity:
            #        max_intensity = intensity
            #        max_wavelength = wavelength
            #ax.set_title("%s, Lmax=%.2f" % (f,max_wavelength))
            ax.set_title("%s" % (f))
            ax.set_xlabel('wavelength, nm')
            ax.set_ylabel('intensity')

            ## Nomenclature is: VHigh→VLow (V'→V")
            ### N2 2nd positive, V'=0
            vcolor = 'red'
            vtext = 1900
            vline = vtext + 20
            ax.vlines(337.0,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(333.0,vtext,"N₂ 2nd⁺(0→0)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(357.6,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(353.6,vtext,"N₂ 2nd⁺(0→1)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(380.4,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(376.4,vtext,"N₂ 2nd⁺(0→2)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(405.8,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(401.8,vtext,"N₂ 2nd⁺(0→3)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(434.3,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(430.3,vtext,"N₂ 2nd⁺(0→4)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(465.5,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(461.5,vtext,"N₂ 2nd⁺0←5)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(503.2,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(499.2,vtext,"N₂ 2nd⁺(0←6)",fontsize=7,color=vcolor,rotation=90)

            ### N2 2nd positive, V'=1
            vcolor = 'green'
            vtext = 1600
            vline = vtext + 20
            ax.vlines(315.8,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(311.8,vtext,"N₂ 2nd⁺(1→0)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(333.8,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(329.8,vtext,"N₂ 2nd⁺(1→1)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(353.6,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(349.6,vtext,"N₂ 2nd⁺(1→2)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(375.4,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(371.4,vtext,"N₂ 2nd⁺(1→3)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(399.7,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(395.7,vtext,"N₂ 2nd⁺(1→4)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(426.8,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(422.8,vtext,"N₂ 2nd⁺(1→5)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(457.3,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(453.3,vtext,"N₂ 2nd⁺(1→6)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(491.7,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(487.7,vtext,"N₂ 2nd⁺(1→7)",fontsize=7,color=vcolor,rotation=90)

            ### N2 2nd positive, V'=2
            vcolor = 'blue'
            vtext = 1300
            vline = vtext + 20
            #ax.vlines(297.6,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(293.6,vtext,"N₂ 2nd⁺(2→0)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(313.5,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(309.5,vtext,"N₂ 2nd⁺(2→1)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(330.9,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(326.9,vtext,"N₂ 2nd⁺(2→2)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(349.9,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(345.9,vtext,"N₂ 2nd⁺(2→3)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(370.9,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(366.9,vtext,"N₂ 2nd⁺(2→4)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(394.2,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(390.2,vtext,"N₂ 2nd⁺(2→5)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(420.0,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(416.0,vtext,"N₂ 2nd⁺(2→6)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(448.9,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(444.9,vtext,"N₂ 2nd⁺(2→7)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(481.3,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(477.3,vtext,"N₂ 2nd⁺(2→8)",fontsize=7,color=vcolor,rotation=90)

            vcolor = 'orange'
            vtext = 1000
            vline = vtext + 20
            #ax.vlines(281.8,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(277.8,vtext,"N₂ 2nd⁺(3→0)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(296.1,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(292.1,vtext,"N₂ 2nd⁺(3→1)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(311.5,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(307.5,vtext,"N₂ 2nd⁺(3→2)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(328.4,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(324.4,vtext,"N₂ 2nd⁺(3→3)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(346.8,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(342.8,vtext,"N₂ 2nd⁺(3→4)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(367.1,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            ax.text(363.1,vtext,"N₂ 2nd⁺(3→5)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(389.4,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(385.4,vtext,"N₂ 2nd⁺(3→6)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(414.0,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(410.0,vtext,"N₂ 2nd⁺(3→7)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(441.5,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(437.5,vtext,"N₂ 2nd⁺(3→8)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(472.2,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(468.2,vtext,"N₂ 2nd⁺(3→9)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(506.7,0,vline,color=vcolor,linestyle='-',linewidth=0.5)
            #ax.text(502.7,vtext,"N₂ 2nd⁺(3→10)",fontsize=7,color=vcolor,rotation=90)

            ### N2 1st negative, V'=0
            vcolor = 'red'
            vtext = 1900
            vline = vtext + 20
            ax.vlines(391.2,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            ax.text(387.2,vtext,"N₂ 1st⁻(0→0)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(427.5,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            ax.text(423.5,vtext,"N₂ 1st⁻(0→1)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(470.6,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            ax.text(466.6,vtext,"N₂ 1st⁻(0→2)",fontsize=7,color=vcolor,rotation=90)

            ### N2 1st negative, V'=1
            vcolor = 'green'
            vtext = 1600
            vline = vtext + 20
            #ax.vlines(358.0,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            #ax.text(354.0,vtext,"N₂ 1st⁻(1→0)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(388.2,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            ax.text(384.2,vtext,"N₂ 1st⁻(1→1)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(423.4 ,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            ax.text(419.4 ,vtext,"N₂ 1st⁻(1→2)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(464.9 ,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            ax.text(460.9 ,vtext,"N₂ 1st⁻(1→3)",fontsize=7,color=vcolor,rotation=90)

            ### N2 1st negative, V'=2
            vcolor = 'blue'
            vtext = 1300
            vline = vtext + 20
            #ax.vlines(330.5,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            #ax.text(326.5,vtext,"N₂ 1st⁻(2→0)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(356.1,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            #ax.text(352.1,vtext,"N₂ 1st⁻(2→1)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(385.5,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            ax.text(381.5,vtext,"N₂ 1st⁻(2→2)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(419.7,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            ax.text(415.7,vtext,"N₂ 1st⁻(2→3)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(459.7,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            ax.text(455.7,vtext,"N₂ 1st⁻(2→4)",fontsize=7,color=vcolor,rotation=90)

            ### N2 1st negative, V'=3
            vcolor = 'orange'
            vtext = 1000
            vline = vtext + 20
            #ax.vlines(329.6,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            #ax.text(325.6,vtext,"N₂ 1st⁻(3→1)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(354.6,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            #ax.text(350.6,vtext,"N₂ 1st⁻(3→2)",fontsize=7,color=vcolor,rotation=90)
            #ax.vlines(383.3,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            #ax.text(379.3,vtext,"N₂ 1st⁻(3→3)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(416.5,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            ax.text(412.5,vtext,"N₂ 1st⁻(3→4)",fontsize=7,color=vcolor,rotation=90)
            ax.vlines(455.2,0,vline,color=vcolor,linestyle='--',linewidth=0.5)
            ax.text(451.2,vtext,"N₂ 1st⁻(3→5)",fontsize=7,color=vcolor,rotation=90)

            ax.set_ylim(0,2000)
            ax.set_xlim(275,500)
            fig.savefig(os.path.join(dplot,'spectra',f),dpi=300)
            plt.close(fig)

def listfonts():
    import matplotlib.font_manager
    list = matplotlib.font_manager.get_fontconfig_fonts()
    names = [matplotlib.font_manager.FontProperties(fname=fname).get_name() for fname in list]
    print names

In [86]:
#prepTemperatureData()
#prepPowerData()
#plotTemperatureData(setpoint=40)
#plotPowerData(setpoint=40)

In [87]:
plotspectra()